# Calculate cross-fed metabolites #

In [1]:
import multiprocessing

multiprocessing.set_start_method('spawn')

import pycomo
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import warnings
import os

sb.set_theme()

2025-11-17 14:46:21,452 - pycomo - INFO - Logger initialized.
2025-11-17 14:46:21,455 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:46:21,455 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:46:21,490 - pycomo - INFO - Multiprocess Logger initialized.


In [2]:
pycomo.configure_logger(level="warning")

In [3]:
model_path = "data/community_models/M. smithii_B. thetaiotaomicron_ms_open_min_med_tp_10.xml"
model = pycomo.CommunityModel.load(model_path)

In [4]:
member_names = model.get_member_names()
model.convert_to_fixed_abundance()

growth_df = model.max_growth_rate(sensitivity=6, return_abundances=True)
growth_df.to_csv(f"data/flux_vectors/max_growth_m_smithii_btim_msmithii_min_med_tp_10.csv")



2025-11-17 14:46:41,598 - pycomo - INFO - Logger initialized.
2025-11-17 14:46:41,599 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:46:41,599 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:46:41,599 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:46:44,075 - pycomo - INFO - Logger initialized.
2025-11-17 14:46:44,076 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:46:44,076 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:46:44,077 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:46:52,664 - pycomo - INFO - Logger initialized.
2025-11-17 14:46:52,664 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:46:52,665 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:46:52,665 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:46:54,594 - pycomo - INFO - Logger initialized.
2025-11-17 14:46:54,595 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:46:54,595

In [5]:
growth_df["mean"] = (growth_df["min_flux"] + growth_df["max_flux"]) / 2.
growth_df

,reaction_id,min_flux,max_flux,mean
0,ABYW00000000_1_ms_medium_gapseq_open_fraction_...,0.478641,0.478980,0.478811
1,NC_004663_1_ms_b_tim_medium_gapseq_open_fracti...,0.521020,0.521359,0.521189
2,community_biomass,0.851182,0.851182,0.851182


In [6]:
growth_df.loc[:1,["reaction_id","mean"]].set_index("reaction_id").to_dict()["mean"]

{'ABYW00000000_1_ms_medium_gapseq_open_fraction_reaction': 0.47881050001908615,
 'NC_004663_1_ms_b_tim_medium_gapseq_open_fraction_reaction': 0.521189499980949}

In [7]:
abd_df = growth_df.loc[:1,["reaction_id","mean"]]
abd_df["member"] = abd_df["reaction_id"].apply(lambda x: x.replace("_fraction_reaction",""))
abundances = abd_df.loc[:1,["member","mean"]].set_index("member").to_dict()["mean"]
model.convert_to_fixed_abundance()
model.apply_fixed_abundance(abundances)

sol = model.model.optimize()
mu = sol.objective_value if sol.status != "infeasible" else 0.
model.fba_solution_flux_vector(file_path=f"data/flux_vectors/fba_max_growth_m_smithii_btim_msmithii_min_med_tp_10.csv")
exchg_df, fva_df = model.potential_metabolite_exchanges(fba=False, 
                                                        composition_agnostic=False,
                                                        fraction_of_optimum=1., 
                                                        loopless=True, 
                                                        processes=8, 
                                                        return_flux_vector=True)
exchg_df.to_csv(f"data/flux_vectors/fva_cf_max_growth_m_smithii_btim_msmithii_min_med_tp_10.csv")
fva_df.to_csv(f"data/flux_vectors/fva_max_growth_m_smithii_btim_msmithii_min_med_tp_10.csv")

2025-11-17 14:57:41,247 - pycomo - INFO - Logger initialized.
2025-11-17 14:57:41,249 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:57:41,250 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:57:41,252 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:57:42,965 - pycomo - INFO - Logger initialized.
2025-11-17 14:57:42,966 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:57:42,966 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:57:42,967 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:57:45,533 - pycomo - INFO - Logger initialized.
2025-11-17 14:57:45,534 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:57:45,534 - pycomo - INFO - Utils Logger initialized.
2025-11-17 14:57:45,535 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 14:57:47,567 - pycomo - INFO - Logger initialized.
2025-11-17 14:57:47,568 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 14:57:47,568

In [8]:
member_names

['ABYW00000000_1_ms_medium_gapseq_open',
 'NC_004663_1_ms_b_tim_medium_gapseq_open']

In [9]:
model.convert_to_fixed_abundance()

for i in [0.1, 0.25, 0.75, 0.9]:
    abundances = {member_names[0]: i, member_names[1]: 1.-i}
    model.apply_fixed_abundance(abundances)
    
    sol = model.model.optimize()
    mu = sol.objective_value if sol.status != "infeasible" else 0.
    model.fba_solution_flux_vector(file_path=f"data/flux_vectors/fba_{i}_m_smithii_btim_msmithii_min_med_tp_10.csv")
    exchg_df, fva_df = model.potential_metabolite_exchanges(fba=False, 
                                                            composition_agnostic=False,
                                                            fraction_of_optimum=1., 
                                                            loopless=True, 
                                                            processes=8, 
                                                            return_flux_vector=True)
    exchg_df.to_csv(f"data/flux_vectors/fva_cf_{i}_m_smithii_btim_msmithii_min_med_tp_10.csv")
    fva_df.to_csv(f"data/flux_vectors/fva_{i}_m_smithii_btim_msmithii_min_med_tp_10.csv")

2025-11-17 15:10:49,373 - pycomo - INFO - Logger initialized.
2025-11-17 15:10:49,374 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 15:10:49,374 - pycomo - INFO - Utils Logger initialized.
2025-11-17 15:10:49,376 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 15:10:51,032 - pycomo - INFO - Logger initialized.
2025-11-17 15:10:51,033 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 15:10:51,033 - pycomo - INFO - Utils Logger initialized.
2025-11-17 15:10:51,033 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 15:10:53,156 - pycomo - INFO - Logger initialized.
2025-11-17 15:10:53,156 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 15:10:53,157 - pycomo - INFO - Utils Logger initialized.
2025-11-17 15:10:53,157 - pycomo - INFO - Multiprocess Logger initialized.
2025-11-17 15:10:54,968 - pycomo - INFO - Logger initialized.
2025-11-17 15:10:54,969 - pycomo - INFO - Process Pool Logger initialized.
2025-11-17 15:10:54,969